In [ ]:
#importing the needed libraries

import json
import re

In [ ]:
# Open file in read mode

with open('Task.txt', 'r') as file:

    # Read entire content of the file
    file_content = file.read()
    print(file_content)

In [ ]:
# this fuction parses the content of the file extract the diserable data
def extract_questions(text):
    extracted_questions = []
    current_question = {}
    option_number = 1
    solution_text = ""
    question_captured = False
    solution_captured = False

    for line in text.split("\n"):
        if re.match(r'^\s*Question ID:|\\section\*{Question ID:', line):
            # Start processing a new question
            if current_question:
                if "questionId" in current_question:
                    # Finalize the current question and add to the list
                    current_question["solutionText"] = solution_text.strip().replace("\\\\", "\\")
                    current_question["questionText"] = current_question["questionText"].rstrip('\n')
                    extracted_questions.append(current_question)
                current_question = {}
                solution_text = ""
                option_number = 1
            current_question["questionNumber"] = len(extracted_questions) + 1
            current_question["questionId"] = int(re.search(r"\d+", line).group())
            question_captured = True
            solution_captured = False
        elif re.match(r'^\s*\(A\)|\(B\)|\(C\)|\(D\)', line):
            # Process options for the current question
            question_captured = False
            current_question.setdefault("options", []).append({"optionNumber": option_number, "optionText": line[4:].strip(), "isCorrect": False})
            option_number += 1
        elif re.match(r'Answer \(([A-D])\)|\\section\*{Answer \(([A-D])\)}', line):
            # Identify the correct option
            match = re.search(r'Answer \(([A-D])\)|\\section\*{Answer \(([A-D])\)}', line)
            if match:
                correct_option = match.group(1) or match.group(2)
                if current_question:
                    for option in current_question["options"]:
                        option["isCorrect"] = option["optionNumber"] == ord(correct_option) - ord('A') + 1
        elif line.startswith("Sol."):
            # Start capturing solution text
            solution_captured = True
            solution_text += line[len("Sol."):].strip()
        elif solution_captured :
            # Continue capturing solution text
            if line.strip() == "":
                if solution_text.strip() != "":
                    solution_text += "\n"
            else:
                solution_text += line + " "
        elif question_captured:
            # Continue capturing question text
            if current_question.get("questionText") is None:
                current_question["questionText"] = ""
            if line.strip() != "":
                current_question["questionText"] += line.replace("\\", "\\\\").replace("\\\\", "\\").replace(" \\\\ ", " \\ ").strip() + "\n"

    # Finalize the last question if exists
    if current_question and "questionId" in current_question:
        current_question["solutionText"] = solution_text.strip().replace("\\\\", "\\")
        current_question["questionText"] = current_question["questionText"].rstrip('\n')
        extracted_questions.append(current_question)

    return extracted_questions



In [ ]:
# Creating a json file and pushing question data into the file

questions = extract_questions(file_content)
json_data = json.dumps(questions, indent=2)  #convert_to_json(questions)
with open("Result.json", "w") as file:
    file.write(json_data)